In [2]:
import os

import tqdm

os.chdir("/home/anutka/PycharmProjects/Recsys-course-homework_2022")

import xlearn as xl
from recommenders.datasets.pandas_df_utils import LibffmConverter

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

print_n = lambda x: print(f"{x:_}")

import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)
pd.options.mode.chained_assignment = None

In [2]:
print_n(15821472)

15_821_472


## Read data

In [3]:
%%time
df = pd.read_csv('./data/data.csv', parse_dates=["date_time"], date_parser=pd.to_datetime)
not_needed = ["banner_id0", "banner_id1", "rate0",
              "rate1", "g0", "g1", "coeff_sum0", "coeff_sum1", "impressions"]
df.drop(labels=not_needed, axis=1, inplace=True)
df.head()

CPU times: user 32.8 s, sys: 3.81 s, total: 36.6 s
Wall time: 37 s


,date_time,zone_id,banner_id,oaid_hash,campaign_clicks,os_id,country_id,clicks
0,2021-09-27 00:01:30,0,0,5664530014561852622,0,0,0,1
1,2021-09-26 22:54:49,1,1,5186611064559013950,0,0,1,1
2,2021-09-26 23:57:20,2,2,2215519569292448030,3,0,0,1
3,2021-09-27 00:04:30,3,3,6262169206735077204,0,1,1,1
4,2021-09-27 00:06:21,4,4,4778985830203613115,0,1,0,1


Обработку данных я оставлю такой же кaк в предыдущей домашке, за исключением oaid_hash. Если посмотреть на распределение колва наблюдений по юзерам станет понятно, что у нас в выборке примерно процентов 70 юзеров встречаются только один или два раза, таким юзерам я планирую присвоить одну категорию, так как много информации мы не потеряем, но при этом размерность сократим значительно и ускорим вычисления


In [4]:
user_counts = df["oaid_hash"].value_counts()
user_counts.value_counts()

1       4182814
2       1049124
3        432171
4        233187
5        144499
         ...   
480           1
482           1
486           1
493           1
5243          1
Name: oaid_hash, Length: 576, dtype: int64

In [5]:
rare_users = user_counts[user_counts <= 2].index.values
rare_users

array([1917392411660372827, 9079324665557264455, 6791436194155659944, ...,
       8578082337233412909, 8779980718415233023,  453968700792456599])

In [6]:
mask = df['oaid_hash'].isin(rare_users)
n_rare_users = len(rare_users)
n_all_users = len(user_counts)

print_n(n_rare_users)
print_n(n_all_users)
print_n(n_all_users - n_rare_users)

5_231_938
6_510_316
1_278_378


In [7]:
df['oaid_hash'] = np.where(mask, 0, df['oaid_hash'])

In [8]:
df.head()

,date_time,zone_id,banner_id,oaid_hash,campaign_clicks,os_id,country_id,clicks
0,2021-09-27 00:01:30,0,0,0,0,0,0,1
1,2021-09-26 22:54:49,1,1,0,0,0,1,1
2,2021-09-26 23:57:20,2,2,2215519569292448030,3,0,0,1
3,2021-09-27 00:04:30,3,3,0,0,1,1,1
4,2021-09-27 00:06:21,4,4,0,0,1,0,1


In [9]:
# Delete
index = df[(df.date_time.dt.day == 1) & (df.date_time.dt.month == 9)].index
df.drop(index, inplace=True)
print(f"Dataset samples {len(df):_}")  # 15 821 472
df["day"] = df["date_time"].dt.dayofweek
# df['log_clicks'] = np.log(df['campaign_clicks'] + 1e-5)
df.head()

Dataset samples 15_821_471


,date_time,zone_id,banner_id,oaid_hash,campaign_clicks,os_id,country_id,clicks,day
0,2021-09-27 00:01:30,0,0,0,0,0,0,1,0
1,2021-09-26 22:54:49,1,1,0,0,0,1,1,6
2,2021-09-26 23:57:20,2,2,2215519569292448030,3,0,0,1,6
3,2021-09-27 00:04:30,3,3,0,0,1,1,1,0
4,2021-09-27 00:06:21,4,4,0,0,1,0,1,0


In [10]:
train_set = df[df["date_time"].dt.date <= pd.to_datetime('2021-09-30')]
val_set = df[df["date_time"].dt.date == pd.to_datetime('2021-10-01')]
test_set = df[df["date_time"].dt.date == pd.to_datetime('2021-10-02')]
print_n(len(train_set))
print_n(len(val_set))
print_n(len(test_set))

12_049_045
1_643_448
2_128_978


In [ ]:
train_set.head()

In [ ]:
not_needed = ["date_time", "campaign_clicks"]
train_set.drop(labels=not_needed, axis=1, inplace=True)
train_set.head()
train_set.to_csv("data/prep_train.csv", index=False)

In [17]:
not_needed = ["date_time", "campaign_clicks"]
val_set.drop(labels=not_needed, axis=1, inplace=True)
val_set.to_csv("data/prep_val.csv")

In [18]:
not_needed = ["date_time", "campaign_clicks"]
test_set.drop(labels=not_needed, axis=1, inplace=True)
test_set.to_csv("data/prep_test.csv")

In [22]:
del train_set
del val_set
del test_set

In [4]:
train_file = "task_2_fm/train.txt"
valid_file = "task_2_fm/valid.txt"
test_file = "task_2_fm/test.txt"
model_file = "task_2_fm/model.out"
output_file = "task_2_fm/output.txt"

In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


'''
Another CTR comp and so i suspect libffm will play its part, after all it is an atomic bomb for this kind of stuff.
A sci-kit learn inspired script to convert pandas dataframes into libFFM style data.

The script is fairly hacky (hey thats Kaggle) and takes a little while to run a huge dataset.
The key to using this class is setting up the features dtypes correctly for output (ammend transform to suit your needs)

Example below

'''

category_column = ['oaid_hash', 'banner_id', 'clicks', 'zone_id', 'os_id', 'country_id']
class FFMFormatPandas:
#https://github.com/Bobe24/Dataframe2libffm/tree/3e34cb0c195242560d85753b2963ad845691e14e

    def __init__(self):
        self.field_index_ = None
        self.feature_index_ = None
        self.y = None

    def fit(self, df, y=None):
        self.y = y
        df_ffm = df[df.columns.difference([self.y])]
        if self.field_index_ is None:
            self.field_index_ = {col: i for i, col in enumerate(df_ffm)}

        if self.feature_index_ is not None:
            last_idx = max(list(self.feature_index_.values()))

        if self.feature_index_ is None:
            self.feature_index_ = dict()
            last_idx = 0

        for col in df_ffm.columns:
            vals = df_ffm[col].unique()
            for val in vals:
                if pd.isnull(val):
                    continue
                name = '{}_{}'.format(col, val)
                if name not in self.feature_index_:
                    self.feature_index_[name] = last_idx
                    last_idx += 1
            self.feature_index_[col] = last_idx
            last_idx += 1
        return self

    def fit_transform(self, df, y=None):
        self.fit(df, y)
        return self.transform(df)

    def transform_row_(self, row):
        ffm = []
        if self.y != None:
            ffm.append(str(row.loc[row.index == self.y][0]))
        if self.y is None:
            ffm.append(str(0))

        for col, val in row.loc[row.index != self.y].to_dict().items():
            name = '{}_{}'.format(col, val)
            if col in category_column:
                ffm.append('{}:{}:1'.format(self.field_index_[col],
                                            self.feature_index_[name]))
            else:
                ffm.append('{}:{}:{}'.format(self.field_index_[col],
                                             self.feature_index_[col], val))
        return ' '.join(ffm)

    def transform(self, df):
        return pd.Series(
            {idx: self.transform_row_(row) for (idx, row) in enumerate(df.iterrows())})

In [ ]:
converter = FFMFormatPandas()
train_set = pd.read_csv("data/prep_train.csv", index_col=0)
transformed_train = converter.fit_transform(train_set, y='clicks')

In [6]:
transformed_train.to_csv(train_file, index=False, header=False)

In [ ]:
val_set = pd.read_csv("data/prep_val.csv", index_col=0)
transformed_val = converter.fit_transform(val_set, y='clicks')
transformed_val.to_csv(valid_file, index=False, header=False)

In [ ]:
test_set = pd.read_csv("data/prep_test.csv", index_col=0)
transformed_test = converter.fit_transform(test_set, y='clicks')
transformed_test.to_csv(test_file, index=False, header=False)

In [5]:
from recommenders.utils.timer import Timer
from recommenders.tuning.parameter_sweep import generate_param_grid
from sklearn.metrics import roc_auc_score

param = {
    'task': 'binary',
    'lr': 0.1,
    'lambda':0.0001,
    'k': 4,
    'metric': 'auc',
    'epoch': 15
}
auc_scores = []
with Timer() as time_tune:
    ffm_model = xl.create_ffm()
    ffm_model.setTrain(train_file)
    ffm_model.setValidate(valid_file)
    ffm_model.fit(param, model_file)

    ffm_model.setTest(test_file)
    ffm_model.setSigmoid()
    ffm_model.predict(model_file, output_file)

    with open(output_file) as f:
        predictions = f.readlines()

    with open(test_file) as f:
        truths = f.readlines()

    truths = np.array([float(truth.split(' ')[0]) for truth in truths])
    predictions = np.array([float(prediction.strip('')) for prediction in predictions])

    auc_scores.append(roc_auc_score(truths, predictions))

----------------------------------------------------------------------------------------------
           _
          | |
     __  _| |     ___  __ _ _ __ _ __
     \ \/ / |    / _ \/ _` | '__| '_ \ 
      >  <| |___|  __/ (_| | |  | | | |
     /_/\_\_____/\___|\__,_|_|  |_| |_|

        xLearn   -- 0.40 Version --
----------------------------------------------------------------------------------------------

[------------] xLearn uses 12 threads for training task.
[ ACTION     ] Read Problem ...
[------------] First check if the text file has been already converted to binary format.
[------------] Binary file (task_2_fm/train.txt.bin) found. Skip converting text to binary.
[------------] First check if the text file has been already converted to binary format.
[------------] Binary file (task_2_fm/valid.txt.bin) found. Skip converting text to binary.
[------------] Number of Feature: 1175471
[------------] Number of Field: 6
[------------] Time cost for reading problem: 2.89 (sec)
[ A

В процессе работы пришлось пожертвовать фичей campagn clicks, но даже в таких условиях результат получился лучше, чем у линейной модели
